In [1]:
%load_ext lab_black

In [2]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [3]:
import pandas as pd
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.max_columns", None)

#### comparing eu and germany gas storage since start of 2022

In [5]:
eu_gas_storage = pd.read_csv(
    "data/raw/EU-StorageData_GIE_2022-01-01_2022-08-15.csv",
    sep=";",
    parse_dates=["Gas Day Start"],
)

In [6]:
ger_gas_storage = pd.read_csv(
    "data/raw/germany-StorageData_GIE_2022-01-01_2022-08-15.csv",
    sep=";",
    parse_dates=["Gas Day Start"],
)

In [7]:
eu_storage_lite = eu_gas_storage[["Gas Day Start", "Gas in storage (TWh)"]].rename(
    columns={"Gas Day Start": "date", "Gas in storage (TWh)": "eu_twh_storage"}
)

In [8]:
ger_storage_lite = ger_gas_storage[["Gas Day Start", "Gas in storage (TWh)"]].rename(
    columns={"Gas Day Start": "date", "Gas in storage (TWh)": "ger_twh_storage"}
)

In [9]:
eu_ger_storage = eu_storage_lite.merge(ger_storage_lite, on="date")

In [10]:
dw.add_data(chart_id="k5Euz", data=eu_ger_storage)

<Response [204]>

#### need to figure out best source of flow data

In [11]:
# curl <"https://transparency.entsog.eu/api/v1/aggregateddata.csv?forceDownload=true&delimiter=comma&from=2022-01-01&to=2022-08-16&indicator=Allocation,Physical%20Flow&periodType=day&timezone=CET&limit=-1"> --header "x-key: 46332b2f5408a267187a018440417c83"

In [12]:
raw_flow = pd.read_csv(
    "data/raw/export_aggregateddata_2022_08_17_191403.csv",
    parse_dates=["periodFrom", "periodTo"],
)

In [13]:
raw_flow[raw_flow["countryLabel"] == "Germany"].value_counts("adjacentSystemsLabel")

adjacentSystemsLabel
Storage                     3648
Netherlands                 2182
Final Consumers             2050
Czech                       1617
Production                  1497
Transmission                1368
Austria                     1124
BeLux                        912
Joint Bal Zone DK/SE         696
Switzerland                  684
Poland H-gas                 456
TGPS (YAMAL)                 456
TRF                          456
Storage,Transmission          10
Netherlands,Transmission       7
dtype: int64

In [14]:
raw_flow[
    (raw_flow["countryLabel"] == "Germany") & (raw_flow["directionKey"] == "entry")
]

id  dataSet  \
31     1AggregatesDEDE-THE-----DE-TSO-0007entryTransm...        1   
32     1AggregatesDEDE-THE-----DE-TSO-0007entryTransm...        1   
35     1AggregatesDEDE-THE-----DE-TSO-0010entryStorag...        1   
36     1AggregatesDEDE-THE-----DE-TSO-0010entryTransm...        1   
153    1AggregatesDEDE-THE-----DE-TSO-0001entryProduc...        1   
...                                                  ...      ...   
74426  1AggregatesDEDE-THE-----DE-TSO-0009entryTransm...        1   
74436  1AggregatesDEDE-THE-----DE-TSO-0014entryProduc...        1   
74437  1AggregatesDEDE-THE-----DE-TSO-0014entryStorag...        1   
74442  1AggregatesDEDE-THE-----DE-TSO-0016entryTransm...        1   
74443  1AggregatesDEDE-THE-----DE-TSO-0017entryTransm...        1   

      dataSetLabel      indicator periodType          periodFrom  \
31      Aggregates  Physical Flow        day 2022-01-01 06:00:00   
32      Aggregates  Physical Flow        day 2022-01-01 06:00:00   
35      Aggregates  Physical Flow        day 2022-01-01 06:00:00   
36      Aggregates  Physical Flow        day 2022-01-01 06:00:00   
153     Aggregates  Physical Flow        day 2022-01-01 06:00:00   
...            ...            ...        ...                 ...   
74426   Aggregates  Physical Flow        day 2022-08-16 06:00:00   
74436   Aggregates  Physical Flow        day 2022-08-16 06:00:00   
74437   Aggregates  Physical Flow        day 2022-08-16 06:00:00   
74442   Aggregates  Physical Flow        day 2022-08-16 06:00:00   
74443   Aggregates  Physical Flow        day 2022-08-16 06:00:00   

                 periodTo countryKey countryLabel        bzKey    bzShort  \
31    2022-01-02 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
32    2022-01-02 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
35    2022-01-02 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
36    2022-01-02 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
153   2022-01-02 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
...                   ...        ...          ...          ...        ...   
74426 2022-08-17 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
74436 2022-08-17 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
74437 2022-08-17 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
74442 2022-08-17 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   
74443 2022-08-17 06:00:00         DE      Germany  DE-THE-----  DE THE BZ   

                          bzLong  operatorKey  \
31     German Balancing Zone THE  DE-TSO-0007   
32     German Balancing Zone THE  DE-TSO-0007   
35     German Balancing Zone THE  DE-TSO-0010   
36     German Balancing Zone THE  DE-TSO-0010   
153    German Balancing Zone THE  DE-TSO-0001   
...                          ...          ...   
74426  German Balancing Zone THE  DE-TSO-0009   
74436  German Balancing Zone THE  DE-TSO-0014   
74437  German Balancing Zone THE  DE-TSO-0014   
74442  German Balancing Zone THE  DE-TSO-0016   
74443  German Balancing Zone THE  DE-TSO-0017   

                         operatorLabel        tsoEicCode directionKey  \
31                    Fluxys TENP GmbH  21X000000001133M        entry   
32                    Fluxys TENP GmbH  21X000000001133M        entry   
35                      bayernets GmbH  21X-DE-A-A0A0A-A        entry   
36                      bayernets GmbH  21X-DE-A-A0A0A-A        entry   
153          GASCADE Gastransport GmbH  21X-DE-H-A0A0A-L        entry   
...                                ...               ...          ...   
74426            Open Grid Europe GmbH  21X-DE-C-A0A0A-T        entry   
74436                terranets bw GmbH  21X000000001163D        entry   
74437                terranets bw GmbH  21X000000001163D        entry   
74442  OPAL Gastransport GmbH & Co. KG  21X0000000011845        entry   
74443            NEL Gastransport GmbH  21X000000001143J        entry   

            adjac

#### our world in data gas production and energy consumption data 

In [15]:
energy_consumption_raw = pd.read_csv(
    "data/raw/germany-energy-consumption-by-source-and-region.csv"
)

In [16]:
dw_energy_consumption = energy_consumption_raw[
    ((energy_consumption_raw["Code"]) == "DEU")
].drop(columns=["Entity", "Code"])

In [17]:
dw_energy_consumption.columns = (
    dw_energy_consumption.columns.str.replace(r" \(.*\)", "")
    .str.replace("-", "")
    .str.replace("zero filled", "")
    .str.replace("Total zero filled", "")
    .str.replace("TWh", "")
    .str.replace(" ", "_")
    .str.lower()
)

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_17131/1066677427.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dw_energy_consumption.columns.str.replace(r" \(.*\)", "")


In [18]:
dw_energy_consumption.columns

Index(['year', 'biofuels_consumption____total', 'solar_consumption__',
       'wind_consumption__', 'hydro_consumption__', 'nuclear_consumption__',
       'gas_consumption__', 'coal_consumption__', 'oil_consumption__',
       'geo_biomass_other__'],
      dtype='object')

In [19]:
dw_energy_consumption["renewables"] = (
    dw_energy_consumption["biofuels_consumption____total"]
    + dw_energy_consumption["solar_consumption__"]
    + dw_energy_consumption["wind_consumption__"]
    + dw_energy_consumption["hydro_consumption__"]
    + dw_energy_consumption["nuclear_consumption__"]
    + dw_energy_consumption["geo_biomass_other__"]
)

In [20]:
dw_energy_consumption = dw_energy_consumption[
    [
        "year",
        "gas_consumption__",
        "coal_consumption__",
        "oil_consumption__",
        "renewables",
    ]
]

In [21]:
dw.add_data(chart_id="KIguu", data=dw_energy_consumption)

<Response [204]>

### share of energy consumption by source

In [22]:
share = pd.read_csv("data/raw/ger-share-energy-source-sub.csv")

In [23]:
share_gas = share[share["Code"] == "DEU"][["Year", "Gas (% equivalent primary energy)"]]

In [24]:
dw.add_data(chart_id="eFNs4", data=share_gas)

<Response [204]>

### polling data
##### https://www.forschungsgruppe.de/Umfragen/Politbarometer/Archiv/Politbarometer_2022/Juli_II_2022/
##### https://www.forschungsgruppe.de/Umfragen/Politbarometer/Archiv/Politbarometer_2022/April_II_2022/
##### https://www.forschungsgruppe.de/Umfragen/Politbarometer/Archiv/Politbarometer_2022/Maerz_2022/